In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout

## Prepare Input Data
- Train:
    - format: (samples, 234, 130, 1)
- Label:
    - format: (samples, one-hot encode)

In [ ]:
from PIL import Image, ImageOps

# Image.open return RGB
def load_file(path):
    #print(path)
    im = Image.open(path)
    if im is None:
        return None
    
    delta_w = 130 - im.size[0]
    delta_h = 234 - im.size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    new_im = ImageOps.expand(im, padding)
    #new_im.show()

    np_im = np.asarray(new_im)
    return np_im

In [ ]:
def load_data(DIR):
    X_train = None
    Y_train = None
    
    for root, subdirs, files in os.walk(DIR):
        print("dir: ", root)
        file_num = 0
        invalid_file = 0
        
        for filename in files:
            file_num += 1
            #print(filename)
            file_path = os.path.join(root, filename)
            #img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)

            #np_img = np.array(img)
            np_img = load_file(file_path)
            #print(np_img.shape)

            # X_train
            if X_train is None:
                # np_img: (234, 130, 3)
                # X_train: (1, 234, 130, 3)
                X_train = np_img[np.newaxis,:]
            else:
                if (np_img.shape != (234, 130, 3)):
                    #print(file_path + " is invalid")
                    invalid_file += 1
                    continue
                
                # after append:  (sample, 234, 130, 3)
                X_train = np.append(X_train, np_img[np.newaxis,:], axis = 0)

            # Y_train
            y = (int)(root.split("/")[1])
            if Y_train is None:
                Y_train = np.array(y)
            else:
                Y_train = np.append(Y_train, y)

        #print("In dir: ", root, "file num = ", file_num)
        #print("In dir: ", root, "invalid file = ", invalid_file)
        #print("In dir: ", root, "valid file = ", file_num - invalid_file)
    return (X_train, Y_train)

## Loads Train and Test Image 

In [ ]:
# Train
(X_train, Y_train) = load_data('dataset/')
# use one channel
X_train = X_train[:, :, :, 1]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
Y_train = to_categorical(Y_train)
#print(Y_train.shape)

# Test
(X_test, Y_test) = load_data('dataset_test/')
Y_test_bk = Y_test.copy() # Y_test_bk is used in the confusion matrix

# use one channel
X_test = X_test[:, :, :, 1]
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
Y_test = to_categorical(Y_test)

## Model

In [ ]:
model = Sequential()

# 16 (5x5 Conv2D)
model.add(Conv2D(16, kernel_size=(5, 5), padding="same",
                 input_shape=(234, 130, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(32, kernel_size=(5, 5), padding="same",
                 activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))

# The final output number should match the label classes
num = Y_train.shape[1]
model.add(Dense(num, activation="softmax"))

model.summary()

## Compile Model

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])

## Train

In [ ]:
print(X_train.shape)
print(Y_train.shape)
history = model.fit(X_train, Y_train, epochs=20, batch_size=128, verbose=2)

## Save Model

In [ ]:
print("Saving Model: mnist.h5 ...")
model.save("mnist.h5")

## Evaluation

In [ ]:
print("Testing ...")
loss, accuracy = model.evaluate(X_train, Y_train)
print("Accuracy on Training Set = {:.2f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test)
print("Accuracy on Test Set = {:.2f}".format(accuracy))

## Show on graph

In [ ]:
import matplotlib.pyplot as plt

loss = history.history["loss"]
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, "bo-", label="Training Loss")
plt.title("Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
acc = history.history["acc"]
epochs = range(1, len(acc)+1)
plt.plot(epochs, acc, "bo-", label="Training Acc")
plt.title("TrainingAccuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

## Predict

In [ ]:
Y_pred = model.predict_classes(X_test)

## Confusion Matrix

In [ ]:
tb = pd.crosstab(Y_test_bk.astype(int), Y_pred.astype(int),
                 rownames=["label"], colnames=["predict"])
print(tb)
#tb.to_html("Confusion Matrix.html")